In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [100]:
# 공통 패키지 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import os
import sys

# src 경로 추가
PROJECT_ROOT = os.path.abspath(os.path.join(".."))
SRC_DIR = os.path.join(PROJECT_ROOT, "src")
if SRC_DIR not in sys.path:
    sys.path.append(SRC_DIR)

# seaborn 스타일 설정
sns.set(style="whitegrid")

# 경로 및 상수 설정
DATA_DIR = "../data/processed"
OUTPUT_DIR = "../data/modeling"
SEOUL_STATION_PATH = os.path.join(DATA_DIR, "seoul_env_station_mapped.csv")
OUTPUT_IMG = "../outputs/image"

os.makedirs(OUTPUT_DIR, exist_ok=True)

# KMeans 설정
KMEANS_MODE = "auto" # auto, manual
KMEANS_MANUAL_K = 5

# XGBoost 설정
XGB_N_ESTIMATORS = 100

# MCLP 설정
COVERAGE_RADIUS = 0.005
FACILITY_LIMIT = 30

# 평가 함수 import
from evaluation.baseline_evaluator import (
    evaluate_existing_stations,
    evaluate_random_installation,
    evaluate_cluster_centers,
    evaluate_mclp_result
)

In [101]:
import pandas as pd

# 데이터 로드
all_df = pd.read_csv("../data/processed/grid_features.csv")  # 전체 격자 (demand_score 포함)
stations = pd.read_csv("../data/processed/charging_stations_seoul_gridded.csv")  # 충전소 (grid_id 포함)

# 전체 격자 수
total_grids = all_df['grid_id'].nunique()

# 설치된 격자 수
installed_grids = stations['grid_id'].nunique()

# 설치 비율
install_ratio = installed_grids / total_grids * 100

# 수요 커버율 계산
covered = all_df[all_df['grid_id'].isin(stations['grid_id'])]
covered_demand = covered['demand_score'].sum()
total_demand = all_df['demand_score'].sum()
coverage_rate = covered_demand / total_demand * 100

# 결과 출력
print("기존 충전소 기반 분석 결과")
print(f"- 전체 서울 격자 수: {total_grids}")
print(f"- 설치 격자 수: {installed_grids}")
print(f"- 서울 전체 기준 설치 비율: {install_ratio:.2f}%")
print(f"- 수요 기준 커버율: {coverage_rate:.2f}%")

기존 충전소 기반 분석 결과
- 전체 서울 격자 수: 6030
- 설치 격자 수: 1753
- 서울 전체 기준 설치 비율: 29.07%
- 수요 기준 커버율: 96.18%


# K-means

In [102]:
from modeling.kmeans_model import run_kmeans

# 데이터 로드
grid = pd.read_csv(f"{DATA_DIR}/grid_system_processed.csv")

# 수요가 가장 높은 클러스터만 추출
grid , used_k = run_kmeans(
    grid,
    mode=KMEANS_MODE,
    # manual_k=KMEANS_MANUAL_K,
    return_top_cluster_only=True
)

Finding optimal k: 100%|██████████| 9/9 [00:00<00:00, 20.61it/s]

[AUTO MODE] 최적 k = 2
Inertia by k: {2: 394878040.6060905, 3: 182752136.79175264, 4: 111630868.37625773, 5: 83130803.13068569, 6: 65292788.38155131, 7: 54917904.7908943, 8: 48181642.39693807, 9: 36633734.731904596, 10: 32347245.131689377}

[Cluster별 평균 수요]
cluster
1.0    1360.575095
0.0      57.366915
Name: demand_score, dtype: float64

[필터링] 수요가 가장 높은 클러스터 (cluster=1.0)만 반환됨.


KMeans로 수요 밀집 지역을 선별한 후,

해당 지역의 feature만 따로 정리해 저장하는 전처리 단계

→ 이후 모델(XGBoost, MCLP)에서 이 subset만 사용할 수 있게 함.

In [103]:
# 필요한 컬럼만 유지 (grid: KMeans 결과)
grid = grid[['grid_id', 'center_lat', 'center_lon', 'cluster']]

# 원본 feature 로드
features_all = pd.read_csv(f"{DATA_DIR}/grid_features.csv")

# 병합 전: features_all 좌표 제거 (혼동 방지)
features_all = features_all.drop(columns=['center_lat', 'center_lon'], errors='ignore')

# 병합
features = features_all.merge(grid, on='grid_id', how='inner')

# cluster 정수형 처리
features['cluster'] = features['cluster'].astype(int)

# 저장 전 좌표 정리: 이미 이름이 정돈돼 있음
# → center_lat, center_lon 이 grid 기준으로만 존재
features = features.loc[:, ~features.columns.duplicated()]  # 혹시 모를 중복 제거

# 저장
features.to_csv(f"{OUTPUT_DIR}/kmeans_grid_features.csv", index=False)
print("좌표 컬럼 중복 없이 정리 및 저장 완료")

# 확인
print(f"사용 가능한 feature 컬럼: {features.columns.tolist()}")

좌표 컬럼 중복 없이 정리 및 저장 완료
사용 가능한 feature 컬럼: ['grid_id', 'demand_score', 'supply_score', 'commercial_count', 'station_count', 'supply_demand_ratio', 'population_density', 'accessibility_score', 'transport_score', 'center_lat', 'center_lon', 'cluster']


In [104]:
import pandas as pd

# 데이터 로드
all_df = pd.read_csv(f"{DATA_DIR}/grid_features.csv")  # 전체 서울 격자
kmeans_df = pd.read_csv(f"{OUTPUT_DIR}/kmeans_grid_features.csv")  # 클러스터 기반 격자

# 전체 서울 격자 수
total_grids = all_df['grid_id'].nunique()

# 클러스터 선택된 격자 수
selected_grids = kmeans_df['grid_id'].nunique()

# 서울 전체 기준 설치 비율
install_ratio = selected_grids / total_grids * 100

# 수요 기준 커버율 (전체 수요 중 클러스터 격자의 수요가 차지하는 비중)
total_demand = all_df['demand_score'].sum()
covered_demand = kmeans_df['demand_score'].sum()
coverage_rate = covered_demand / total_demand * 100

# 출력
print(f"클러스터링 기반 분석 결과")
print(f"- 전체 서울 격자 수: {total_grids}")
print(f"- 클러스터 격자 수: {selected_grids}")
print(f"- 서울 전체 기준 설치 비율: {install_ratio:.2f}%")
print(f"- 수요 기준 커버율: {coverage_rate:.2f}%")

클러스터링 기반 분석 결과
- 전체 서울 격자 수: 6030
- 클러스터 격자 수: 526
- 서울 전체 기준 설치 비율: 8.72%
- 수요 기준 커버율: 69.39%


In [105]:
# 선택시항

from utils.inspect_dataframe import inspect_dataframe
from visualization.map_visualizer import visualize_cluster_map

# 1. kmeans_grid_features.csv 특징
inspect_dataframe(features, name="features")

# 2. kmeans 시각화 - 서울 전체 (회색) - 수요 (빨간색)
visualize_cluster_map(
    all_features_path=f"{DATA_DIR}/grid_features.csv",
    filtered_features_path=f"{OUTPUT_DIR}/kmeans_grid_features.csv",
    output_path=f"../outputs/maps/kmeans_visualization.html"
)

🧾 입력 데이터 이름: features
🔢 shape: (526, 12)
🔑 컬럼 목록:
['grid_id', 'demand_score', 'supply_score', 'commercial_count', 'station_count', 'supply_demand_ratio', 'population_density', 'accessibility_score', 'transport_score', 'center_lat', 'center_lon', 'cluster']

🧾 예시 5개:
        grid_id  demand_score  supply_score  commercial_count  station_count  \
0  GRID_011_036        1372.5      27.15053               200              1   
1  GRID_011_037         911.5      27.15053               200              0   
2  GRID_012_035         883.0      27.15053               200              1   
3  GRID_012_036         952.5      27.15053               200              0   
4  GRID_014_033         946.5      27.15053               200              0   

   supply_demand_ratio  population_density  accessibility_score  \
0            50.551499                2400                  0.0   
1            33.572089                2400                  0.0   
2            32.522385                2400         

# XGBoost

In [106]:
from modeling.xgboost_model import train_and_predict

# 학습 feature
selected_features = [
    'supply_score',
    'station_count',
    'commercial_count',
    'supply_demand_ratio',
    'population_density',
    'accessibility_score',
    'transport_score',
    'cluster'
]

# 범주형 처리
features['cluster'] = features['cluster'].astype('category')

# XGBoost 학습 및 예측
features_with_pred, metrics, model = train_and_predict(
    df=features,
    features=selected_features,
    label='demand_score',
    n_estimators=XGB_N_ESTIMATORS,
    verbose=True
)

# 저장
features_with_pred.to_csv(f"{OUTPUT_DIR}/xgboost_grid_features.csv", index=False)


XGBoost 성능:
MAE: 50.72
RMSE: 81.40
R²: 0.9826


# MCLP

In [15]:
from modeling.mclp_model import run_sensitivity_analysis_parallel

results_df = run_sensitivity_analysis_parallel(
    df=features_with_pred,
    coverage_radii_km=[0.3, 0.4, 0.5, 0.55, 0.6],
    facility_limits=list(range(50, 500, 10)),
    demand_column='predicted_demand_score',
    max_workers=4
)

[1/225 완료] 반경 0.3km, 설치 50개
[2/225 완료] 반경 0.3km, 설치 80개
[3/225 완료] 반경 0.3km, 설치 60개
[4/225 완료] 반경 0.3km, 설치 70개
[5/225 완료] 반경 0.3km, 설치 90개
[6/225 완료] 반경 0.3km, 설치 100개
[7/225 완료] 반경 0.3km, 설치 110개
[8/225 완료] 반경 0.3km, 설치 120개
[9/225 완료] 반경 0.3km, 설치 130개
[10/225 완료] 반경 0.3km, 설치 140개
[11/225 완료] 반경 0.3km, 설치 150개
[12/225 완료] 반경 0.3km, 설치 160개
[13/225 완료] 반경 0.3km, 설치 170개
[14/225 완료] 반경 0.3km, 설치 180개
[15/225 완료] 반경 0.3km, 설치 190개
[16/225 완료] 반경 0.3km, 설치 200개
[17/225 완료] 반경 0.3km, 설치 210개
[18/225 완료] 반경 0.3km, 설치 220개
[19/225 완료] 반경 0.3km, 설치 230개
[20/225 완료] 반경 0.3km, 설치 240개
[21/225 완료] 반경 0.3km, 설치 250개
[22/225 완료] 반경 0.3km, 설치 260개
[23/225 완료] 반경 0.3km, 설치 270개
[24/225 완료] 반경 0.3km, 설치 280개
[25/225 완료] 반경 0.3km, 설치 290개
[26/225 완료] 반경 0.3km, 설치 300개
[27/225 완료] 반경 0.3km, 설치 310개
[28/225 완료] 반경 0.3km, 설치 320개
[29/225 완료] 반경 0.3km, 설치 330개
[30/225 완료] 반경 0.3km, 설치 340개
[31/225 완료] 반경 0.3km, 설치 350개
[32/225 완료] 반경 0.3km, 설치 360개
[33/225 완료] 반경 0.3km, 설치 370개
[34/225 완료] 반경 0.3km, 설치

In [22]:
# 데이터 로드
all_df = pd.read_csv(f"{DATA_DIR}/grid_features.csv")  # 전체 서울 격자
kmeans_df = pd.read_csv(f"{OUTPUT_DIR}/kmeans_grid_features.csv")  # 클러스터 기반 격자

# 전체 서울 격자 수
total_grids = all_df['grid_id'].nunique()

# 각 실험별 계산
results_df['서울 전체 기준 설치 비율'] = results_df['설치 격자 수'] / total_grids * 100

# 컬럼 정리 및 이름 변경 (선택)
results_df = results_df[[
    'coverage_radius_km',
    '설치 격자 수',
    '서울 전체 기준 설치 비율',
    '수요 기준 커버율',
    '설치 1개당 커버 수요'
]]

# 저장
results_df.to_csv("../data/modeling/mclp_summary.csv", index=False)
print("요약 결과 저장 완료.")

요약 결과 저장 완료.


In [18]:
print(results_df.columns.tolist())

['coverage_radius_km', '설치 격자 수', '서울 전체 기준 설치 비율', '수요 기준 커버율', '설치 1개당 커버 수요']


In [80]:
import pandas as pd
from modeling.mclp_model import find_elbow_point, print_elbow_summary

# 요약된 결과 불러오기
results_df = pd.read_csv("../data/modeling/mclp_summary.csv")

# 누락된 컬럼 복원: rename
results_df = results_df.rename(columns={
    '설치 격자 수': 'facility_limit',
    '수요 기준 커버율': 'coverage_rate',
    '설치 1개당 커버 수요': 'demand_satisfaction_ratio'
})

# covered_demand 추정 (설치 수 * 효율)
results_df['covered_demand'] = results_df['facility_limit'] * results_df['demand_satisfaction_ratio']

# 1. Elbow 후보 탐색
elbow_df = find_elbow_point(results_df, threshold_ratio=0.6)

# 2. 최적 지점 및 elbow 지점 요약 출력
best_row = results_df.loc[results_df['coverage_rate'].idxmax()]
elbow_row = elbow_df.iloc[0] if not elbow_df.empty else best_row

# 3. 요약 출력
print_elbow_summary(best_row, elbow_row)
print(f"최적 반경: {best_row['coverage_radius_km']} km")
print(f"elbow_point 반경: {elbow_row['coverage_radius_km']} km")


민감도 분석 결과 요약
 - 최적 설치 수 기준 (best_limit): 490개
   → 커버 수요: 689645.60, 커버율: 96.30%
 - 꺾이는 지점 기준 (elbow_point): 190개
   → 커버 수요: 393330.40, 커버율: 54.93%
최적 반경: 0.3 km
elbow_point 반경: 0.3 km


In [107]:
# 목표 커버율
target_coverage = 69.39

# 조건에 맞는 행 필터링
candidate = results_df[results_df['coverage_rate'] >= target_coverage]

if not candidate.empty:
    optimal_row = candidate.loc[candidate['facility_limit'].idxmin()]
    print("[MCLP 최소 설치 수로 동일 커버율 달성]")
    print(f"- 필요한 설치 수: {optimal_row['facility_limit']}")
    print(f"- 예상 커버율: {optimal_row['coverage_rate']:.2f}%")
    print(f"- 설치 1개당 커버 수요: {optimal_row['demand_satisfaction_ratio']:.2f}")
else:
    print("해당 커버율 이상을 달성한 실험이 없습니다.")

[MCLP 최소 설치 수로 동일 커버율 달성]
- 필요한 설치 수: 280.0
- 예상 커버율: 70.08%
- 설치 1개당 커버 수요: 1792.40


In [108]:
from modeling.mclp_model import solve_mclp

# 1. 설정
COVERAGE_RADIUS_KM = 0.3  # 반경 300m

# 2. MCLP 실행
final_df, final_summary, _ = solve_mclp(
    df=features_with_pred,
    coverage_radius=COVERAGE_RADIUS_KM,  # 단위: km로 통일
    facility_limit=280,
    demand_column='predicted_demand_score',
    verbose=True
)

설치지 수: 280개
커버 수요: 501,872.89 / 총 수요: 716,109.91
커버율: 70.08%


In [109]:
# 3. 결과 저장
final_df.to_csv(f"{OUTPUT_DIR}/mclp_grid_features.csv", index=False)     # 전체 결과
final_df[final_df['selected'] == 1].to_csv(f"{OUTPUT_DIR}/mclp_selected_grid_features.csv", index=False)  # 선택된 설치 위치만 저장

In [112]:
# 선택사항 - mclp 후 지역 시각화

from visualization.map_visualizer import visualize_selected_sites_map

visualize_selected_sites_map(
    features_path=f"{OUTPUT_DIR}/mclp_grid_features.csv",
    output_path=f"../outputs/maps/mclp_result_map.html"
)

설치지 시각화 저장 완료: ../outputs/maps/mclp_result_map.html


# 평가

## 설치 방식 비교 실험

### 1. 실험 목적
전기차 충전소 입지 선정 방식에 따라 **미래 수요 상위 지역을 얼마나 잘 커버하는지**를 비교한다.  
수요 기반 알고리즘(MCLP, 클러스터링 등)이 **기존 설치 방식이나 무작위 설치 대비 얼마나 효율적인지** 확인하는 것이 목적이다.

---

### 2. 실험 방식

- **수요 기준**: `future_demand_gridded.csv`의 adjusted_ev_demand 컬럼 기준
- **상위 격자 수**: 전체 격자 중 상위 20%인 `demand_k = 47`개
- **설치 방식별 선정 방식**:
  - **기존 충전소 전체**: 서울시의 모든 기존 충전소 설치 격자
  - **랜덤 설치**: 기존 충전소 중 무작위 47개 선택
  - **클러스터 기반 설치**: KMeans 클러스터 격자 중 `demand_score` 상위 47개
  - **MCLP 추천 설치**: KMeans → XGBoost → MCLP 로 구성된 복합 전략

---

### 3. 평가 지표

- **포함된 격자 수**: 상위 47개 수요 격자 중 설치된 격자의 수
- **커버 수요**: 해당 격자들이 커버한 수요 총합
- **커버율 (%)**: `(커버 수요 / 전체 상위 수요) * 100`
- **설치 효율**: `커버 수요 / 설치 격자 수`

---

### 4. 실험 결과

| 설치 방식             | 포함 격자 수 | 커버 수요     | 전체 수요     | 커버율 (%) | 설치 효율 |
|----------------------|--------------|---------------|---------------|-------------|------------|
| 기존 충전소 전체      | 33 / 47      | 275,653.50    | 324,565.50    | 84.93       | 157.25     |
| 랜덤 설치            | 0 / 47       | 0.00          | 324,565.50    | 0.00        | 0.00       |
| 클러스터 기반 설치   | 1 / 47       | 6,103.50      | 324,565.50    | 1.88        | 129.86     |
| MCLP 추천 설치       | 4 / 47       | 34,884.00     | 324,565.50    | 10.75       | 742.21     |

---

### 5. 분석 요약

- **기존 충전소 전체**는 높은 커버율을 보이지만 설치 효율은 낮다.
- **랜덤 설치**는 커버 수요가 전무하여 가장 비효율적이다.
- **클러스터 기반 설치**는 예측 수요는 높았지만, 상위 수요 지역과 겹침이 적어 실효성이 낮다.
- **MCLP 추천 설치**는 상대적으로 적은 수의 설치만으로도 높은 효율을 보이며, 전략적 입지 선정의 효과를 입증하였다.

In [113]:
import pandas as pd
from modeling.xgboost_model import train_and_predict

# 1. 데이터 로드 (전체 격자 + 클러스터 없이)
features_all = pd.read_csv("../data/processed/grid_features.csv")

# 2. 사용할 피처 선택
selected_features = [
    'supply_score',
    'station_count',
    'commercial_count',
    'supply_demand_ratio',
    'population_density',
    'accessibility_score',
    'transport_score'
]

# 3. 학습 실행
features_with_pred, metrics, model = train_and_predict(
    df=features_all.copy(),
    features=selected_features,
    label='demand_score',
    n_estimators=100,
    verbose=True
)

# 4. 저장
features_with_pred.to_csv("../data/modeling/xgboost_grid_features_test.csv", index=False)
print("전체 격자 기준 XGBoost 예측 결과 저장 완료.")


XGBoost 성능:
MAE: 12.89
RMSE: 61.18
R²: 0.9798
전체 격자 기준 XGBoost 예측 결과 저장 완료.


In [114]:
import pandas as pd

# 1. 데이터 로드 (원본 코드에서 가져옴)
stations = pd.read_csv("../data/processed/charging_stations_seoul_gridded.csv")

# stations DataFrame에 grid_id 중복이 있는지 확인
total_grid_ids_in_stations = len(stations)
unique_grid_ids_in_stations = stations['grid_id'].nunique()

print(f"원본 'stations' DataFrame의 전체 행 수: {total_grid_ids_in_stations}")
print(f"원본 'stations' DataFrame의 고유한 'grid_id' 수: {unique_grid_ids_in_stations}")

if total_grid_ids_in_stations > unique_grid_ids_in_stations:
    print(f"-> 'stations' DataFrame에 {total_grid_ids_in_stations - unique_grid_ids_in_stations}개의 중복된 'grid_id'가 있습니다.")
else:
    print("-> 'stations' DataFrame에 중복된 'grid_id'가 없습니다.")

# 랜덤 설치 샘플링 로직 테스트
sampled_grids_test = stations.sample(n=526, random_state=42)['grid_id']
unique_sampled_grids_test = set(sampled_grids_test)

print(f"\n랜덤 샘플링된 행 수 (중복 포함): {len(sampled_grids_test)}")
print(f"랜덤 샘플링 후 고유한 grid_id 수: {len(unique_sampled_grids_test)}")

원본 'stations' DataFrame의 전체 행 수: 56351
원본 'stations' DataFrame의 고유한 'grid_id' 수: 1753
-> 'stations' DataFrame에 54598개의 중복된 'grid_id'가 있습니다.

랜덤 샘플링된 행 수 (중복 포함): 526
랜덤 샘플링 후 고유한 grid_id 수: 391


In [123]:
import pandas as pd

# 1. 데이터 로드
stations = pd.read_csv("../data/processed/charging_stations_seoul_gridded.csv")
features_with_pred = pd.read_csv("../data/modeling/xgboost_grid_features_test.csv")
kmeans_features = pd.read_csv("../data/modeling/kmeans_grid_features.csv")
mclp_features = pd.read_csv("../data/modeling/mclp_grid_features.csv")

# 2. 전체 수요 기준 계산
total_demand = features_with_pred['predicted_demand_score'].sum()
print(f"전체 격자 수: {len(features_with_pred)}")
print(f"전체 예측 수요 총합: {total_demand:,.2f}")

# 3. 설치 방식별 격자 선택 (랜덤 설치는 526개로 고정)
strategy_sets = {
    "기존 충전소 전체": set(stations['grid_id']),
    # --- 이 부분을 수정합니다 ---
    "랜덤 설치": set(stations['grid_id'].drop_duplicates().sample(n=526, random_state=42)),
    # --- 수정 끝 ---
    "클러스터 기반 설치": set(kmeans_features['grid_id'].dropna()),
    "MCLP 추천 설치": set(mclp_features[mclp_features['selected'] == 1]['grid_id']),
}

# 4. 평가 함수 (전체 수요 기준 평가)
def evaluate_strategy(label, selected_grids):
    matched_df = features_with_pred[features_with_pred['grid_id'].isin(selected_grids)].copy()
    covered_demand = matched_df['predicted_demand_score'].sum()
    coverage_rate = covered_demand / total_demand * 100
    efficiency = covered_demand / len(selected_grids) if selected_grids else 0

    print(f"\n[{label}]")
    print(f"- 설치 격자 수: {len(selected_grids)}")
    print(f"- 커버 수요: {covered_demand:,.2f}")
    print(f"- 전체 수요: {total_demand:,.2f}")
    print(f"- 커버율: {coverage_rate:.2f}%")
    print(f"- 설치 1개당 커버 수요 (효율): {efficiency:,.2f}")

# 5. 평가 실행
for label, grid_set in strategy_sets.items():
    evaluate_strategy(label, grid_set)

전체 격자 수: 6030
전체 예측 수요 총합: 1,026,048.11

[기존 충전소 전체]
- 설치 격자 수: 1753
- 커버 수요: 978,846.38
- 전체 수요: 1,026,048.11
- 커버율: 95.40%
- 설치 1개당 커버 수요 (효율): 558.38

[랜덤 설치]
- 설치 격자 수: 526
- 커버 수요: 299,406.57
- 전체 수요: 1,026,048.11
- 커버율: 29.18%
- 설치 1개당 커버 수요 (효율): 569.21

[클러스터 기반 설치]
- 설치 격자 수: 526
- 커버 수요: 702,554.73
- 전체 수요: 1,026,048.11
- 커버율: 68.47%
- 설치 1개당 커버 수요 (효율): 1,335.66

[MCLP 추천 설치]
- 설치 격자 수: 280
- 커버 수요: 491,258.41
- 전체 수요: 1,026,048.11
- 커버율: 47.88%
- 설치 1개당 커버 수요 (효율): 1,754.49


In [118]:
uncovered_by_existing = set(features_with_pred['grid_id']) - strategy_sets["기존 충전소 전체"]
# 기존 충전소가 커버하지 못한 격자
uncovered_by_existing = set(features_with_pred['grid_id']) - strategy_sets["기존 충전소 전체"]

# 각 전략이 추가로 커버한 격자
random_new_coverage = strategy_sets["랜덤 설치"] & uncovered_by_existing
cluster_new_coverage = strategy_sets["클러스터 기반 설치"] & uncovered_by_existing
mclp_new_coverage = strategy_sets["MCLP 추천 설치"] & uncovered_by_existing

print(f"랜덤 설치가 새롭게 커버한 격자 수: {len(random_new_coverage)}")
print(f"클러스터 기반 설치가 새롭게 커버한 격자 수: {len(cluster_new_coverage)}")
print(f"MCLP 추천 설치가 새롭게 커버한 격자 수: {len(mclp_new_coverage)}")

def covered_demand(grid_ids):
    return features_with_pred[features_with_pred['grid_id'].isin(grid_ids)]['predicted_demand_score'].sum()

print(f"랜덤 설치가 추가로 커버한 수요: {covered_demand(random_new_coverage):,.2f}")
print(f"클러스터 기반 설치가 추가로 커버한 수요: {covered_demand(cluster_new_coverage):,.2f}")
print(f"MCLP 추천 설치가 추가로 커버한 수요: {covered_demand(mclp_new_coverage):,.2f}")


랜덤 설치가 새롭게 커버한 격자 수: 0
클러스터 기반 설치가 새롭게 커버한 격자 수: 15
MCLP 추천 설치가 새롭게 커버한 격자 수: 7
랜덤 설치가 추가로 커버한 수요: 0.00
클러스터 기반 설치가 추가로 커버한 수요: 18,020.96
MCLP 추천 설치가 추가로 커버한 수요: 11,093.12


In [121]:
import pandas as pd

# 1. 데이터 로드
features_with_pred = pd.read_csv("../data/modeling/xgboost_grid_features_test.csv")
stations = pd.read_csv("../data/processed/charging_stations_seoul_gridded.csv")
kmeans_features = pd.read_csv("../data/modeling/kmeans_grid_features.csv")
mclp_features = pd.read_csv("../data/modeling/mclp_grid_features.csv")

# 2. 전체 수요 계산
total_demand = features_with_pred['predicted_demand_score'].sum()

# 3. 전략별 격자 셋 생성
strategy_sets = {
    "기존 충전소 전체": set(stations['grid_id']),
    "랜덤 설치": set(stations['grid_id'].drop_duplicates().sample(n=526, random_state=42)),
    "클러스터 기반 설치": set(kmeans_features['grid_id'].dropna()),
    "MCLP 추천 설치": set(mclp_features[mclp_features['selected'] == 1]['grid_id']),
}

# 4. 기존이 커버하지 못한 격자
uncovered_by_existing = set(features_with_pred['grid_id']) - strategy_sets["기존 충전소 전체"]

# 5. 각 전략이 새롭게 커버한 격자
random_new = strategy_sets["랜덤 설치"] & uncovered_by_existing
cluster_new = strategy_sets["클러스터 기반 설치"] & uncovered_by_existing
mclp_new = strategy_sets["MCLP 추천 설치"] & uncovered_by_existing

# 6. 각 전략의 추가 수요 합
def covered_demand(grid_ids):
    return features_with_pred[features_with_pred['grid_id'].isin(grid_ids)]['predicted_demand_score'].sum()

print(f"\n[신규 커버 격자 수 및 수요]")
print(f"랜덤 설치: {len(random_new)}개, 수요: {covered_demand(random_new):,.2f}")
print(f"클러스터 기반 설치: {len(cluster_new)}개, 수요: {covered_demand(cluster_new):,.2f}")
print(f"MCLP 추천 설치: {len(mclp_new)}개, 수요: {covered_demand(mclp_new):,.2f}")

# 7. 전체 수요 기준 정렬 및 백분위 계산
sorted_features = features_with_pred.sort_values(by='predicted_demand_score', ascending=False).reset_index(drop=True)
sorted_features['rank'] = sorted_features.index + 1
sorted_features['percentile'] = sorted_features['rank'] / len(sorted_features) * 100

# 8. 신규 커버 격자 각각의 백분위
def get_percentiles(df, grid_ids, label):
    return df[df['grid_id'].isin(grid_ids)][['grid_id', 'predicted_demand_score', 'rank', 'percentile']].assign(strategy=label)

df_cluster = get_percentiles(sorted_features, cluster_new, '클러스터 기반 설치')
df_mclp = get_percentiles(sorted_features, mclp_new, 'MCLP 추천 설치')
df_random = get_percentiles(sorted_features, random_new, '랜덤 설치')

# 9. 결과 통합
final_df = pd.concat([df_cluster, df_mclp, df_random]).sort_values(by='rank').reset_index(drop=True)

# 10. 결과 출력
print("\n[신규 커버 격자 상위 수요 비율 분석]")
print(final_df.to_string(index=False))


[신규 커버 격자 수 및 수요]
랜덤 설치: 0개, 수요: 0.00
클러스터 기반 설치: 15개, 수요: 18,020.96
MCLP 추천 설치: 7개, 수요: 11,093.12

[신규 커버 격자 상위 수요 비율 분석]
     grid_id  predicted_demand_score  rank  percentile   strategy
GRID_039_048             3131.772461    15    0.248756 클러스터 기반 설치
GRID_039_048             3131.772461    15    0.248756 MCLP 추천 설치
GRID_018_043             1656.161133   108    1.791045 클러스터 기반 설치
GRID_018_043             1656.161133   108    1.791045 MCLP 추천 설치
GRID_024_073             1493.979004   137    2.271973 클러스터 기반 설치
GRID_024_073             1493.979004   137    2.271973 MCLP 추천 설치
GRID_035_042             1337.928223   189    3.134328 클러스터 기반 설치
GRID_035_042             1337.928223   189    3.134328 MCLP 추천 설치
GRID_031_066             1330.269287   192    3.184080 클러스터 기반 설치
GRID_031_066             1330.269287   192    3.184080 MCLP 추천 설치
GRID_038_054             1088.735840   265    4.394693 클러스터 기반 설치
GRID_038_054             1088.735840   265    4.394693 MCLP 추천 설치
GRID_022_073      

In [122]:
import folium

# 11. 지도 시각화를 위한 좌표 정보 병합
map_df = final_df.merge(
    features_with_pred[['grid_id', 'center_lat', 'center_lon']],
    on='grid_id',
    how='left'
)

# 12. 지도 객체 생성 (서울 중심)
m = folium.Map(location=[37.55, 126.98], zoom_start=11)

# 13. 마커 색상 지정
color_map = {
    '클러스터 기반 설치': 'blue',
    'MCLP 추천 설치': 'red',
    '랜덤 설치': 'green'
}

# 14. 원 마커 추가
for _, row in map_df.iterrows():
    folium.CircleMarker(
        location=[row['center_lat'], row['center_lon']],
        radius=7,
        color=color_map.get(row['strategy'], 'gray'),
        fill=True,
        fill_color=color_map.get(row['strategy'], 'gray'),
        fill_opacity=0.8,
        popup=folium.Popup(
            f"{row['grid_id']}<br>{row['strategy']}<br>수요: {row['predicted_demand_score']:.1f}<br>상위 {row['percentile']:.2f}%",
            max_width=250
        )
    ).add_to(m)

# 15. 저장
m.save("../outputs/maps/new_covered_grids_map.html")
print("\n지도 파일이 'new_covered_grids_map.html'로 저장되었습니다.")



지도 파일이 'new_covered_grids_map.html'로 저장되었습니다.


# XX (기타)

In [14]:
import pandas as pd

# 파일 경로 지정
paths = {
    "future_demand": "../data/processed/future_demand_gridded_2.csv",
    "grid_features": "../data/processed/grid_features.csv",
    "kmeans_features": "../data/modeling/kmeans_grid_features.csv",
    "mclp_features": "../data/modeling/mclp_selected_grid_features.csv",
    "charging_stations": "../data/processed/charging_stations_seoul_gridded.csv"
}

# 결과 출력
for name, path in paths.items():
    df = pd.read_csv(path)
    n = df['grid_id'].nunique()
    print(f"{name} → grid_id 개수: {n}")

future_demand → grid_id 개수: 397
grid_features → grid_id 개수: 6030
kmeans_features → grid_id 개수: 526
mclp_features → grid_id 개수: 89
charging_stations → grid_id 개수: 1753
